In [6]:
!pip install transformers
!pip install sentencepiece
from transformers import pipeline
from pandas import DataFrame


from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:

import re
from pandas import DataFrame, concat
import os


def preprocess(text_file):
    
    book = text_file
    book = re.split('\d+\.', book)
    bookch=list()
    text=list()
    for index, chapter in enumerate(book, start=0):
        bookch.append(index)
        text.append(chapter)
    return text #for paragraph in text


def SentANAL(text_file_path):
  MODEL =f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

  tokenizer = AutoTokenizer.from_pretrained(MODEL)
  config = AutoConfig.from_pretrained(MODEL)

  model = AutoModelForSequenceClassification.from_pretrained(MODEL)
  model.save_pretrained(MODEL)
  tokenizer.save_pretrained(MODEL)
  config.save_pretrained(MODEL)

  text_file = open(text_file_path,"r",encoding='UTF8').read()
  text = preprocess(text_file)

  fragment_sa=[] #where the output will be stored
  for fragment in text:  ##ITERATION -- apply to each fragment from our list##

    encoded_input = tokenizer(fragment, padding='max_length', truncation=True,max_length=511, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    fragment_sa.append(scores)

  # Print labels and scores --here they are rounded up, but it includes labels and i don't like labels
  # scores=[]
  # ranking = np.argsort(scores)
  # ranking = ranking[::-1]
  # for i in range(scores.shape[0]):
  #     l = config.id2label[ranking[i]]
  #     s = scores[ranking[i]]
  #     scores_final =f"{i+1}) {l} {np.round(float(s), 4)}"
  #     scores_l.append(scores_final)

  sentiment_df = DataFrame(np.vstack(fragment_sa),columns=['Negative','Neutral','Positive'])

  return sentiment_df

def create_df(text_file_path,book_number,sa_to_merge):
    book = open(text_file_path,"r",encoding='UTF8').read()
    book_l=re.split('\d+\.', book)
    bookch=list()
    text=list()
    for index, chapter in enumerate(book_l, start=0):
        bookch.append(index)
        text.append(chapter)
    data={'Book': book_number, 'BookChapter':bookch}
    book_df=DataFrame(data,index=None)
    book_SA_df=concat([book_df,sa_to_merge],axis=1)
    book_SA_df=book_SA_df.iloc[1: , :]
    book_SA_df.reset_index(drop=True)
    return book_SA_df



def store_csv(df_to_store,file_path,file_name):


  f_name= file_name
  df_to_store.to_csv(os.path.join(file_path, (f_name+'.csv')))
  return True



text_file_path='/content/drive/MyDrive/DHDK/SA_Meditations/SA/MAtext/MeditationsBook12.txt'
sentiment_df= SentANAL(text_file_path)
book1_sa_df= create_df(text_file_path, "12", sentiment_df)
print(store_csv(book1_sa_df,'/content/drive/MyDrive/DHDK/SA_Meditations','SA_df_book12'))


True
